# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cape town,-33.9258,18.4232,21.23,82,0,8.23,ZA,1675625196
1,1,saint marys,30.7305,-81.5465,23.03,83,100,1.79,US,1675625435
2,2,sao filipe,14.8961,-24.4956,21.71,62,98,6.29,CV,1675625543
3,3,ostrovnoy,68.0531,39.5131,-14.15,97,92,5.11,RU,1675625544
4,4,victoria,22.2855,114.1577,18.51,89,100,2.24,HK,1675625444


In [18]:
city_data_df.count()

City_ID       561
City          561
Lat           561
Lng           561
Max Temp      561
Humidity      561
Cloudiness    561
Wind Speed    561
Country       557
Date          561
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "StamenTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_data = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] ==0)]

# Drop any rows with null values
narrow_data = narrow_data.dropna()
# Display sample data
narrow_data.count()

City_ID       6
City          6
Lat           6
Lng           6
Max Temp      6
Humidity      6
Cloudiness    6
Wind Speed    6
Country       6
Date          6
dtype: int64

In [21]:
narrow_data.head(6)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,catalina,32.5056,-110.9211,24.15,19,0,1.34,US,1675625389
70,70,saint-pierre,-21.3393,55.4781,26.82,78,0,1.54,RE,1675625567
269,269,inhambane,-23.8650,35.3833,23.78,86,0,3.80,MZ,1675625659
434,434,chokwe,-24.5333,32.9833,22.87,84,0,2.47,MZ,1675625735
472,472,salalah,17.0151,54.0924,23.05,73,0,2.06,OM,1675625747
537,537,nelson bay,-32.7167,152.1500,23.33,93,0,1.34,AU,1675625779


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_data[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = narrow_data.copy()
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,38,catalina,32.5056,-110.9211,24.15,19,0,1.34,US,1675625389,
70,70,saint-pierre,-21.3393,55.4781,26.82,78,0,1.54,RE,1675625567,
269,269,inhambane,-23.8650,35.3833,23.78,86,0,3.80,MZ,1675625659,
434,434,chokwe,-24.5333,32.9833,22.87,84,0,2.47,MZ,1675625735,
472,472,salalah,17.0151,54.0924,23.05,73,0,2.06,OM,1675625747,
537,537,nelson bay,-32.7167,152.1500,23.33,93,0,1.34,AU,1675625779,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"



    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as err:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, f"Hotel Name"] = "No hotel found" + str(err)
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
catalina - nearest hotel: Catalina Inn
saint-pierre - nearest hotel: Tropic Hotel
inhambane - nearest hotel: Hotel de Inhambane
chokwe - nearest hotel: Limpopo Hotel
salalah - nearest hotel: Muscat International Hotel
nelson bay - nearest hotel: No hotel found'name'


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,38,catalina,32.5056,-110.9211,24.15,19,0,1.34,US,1675625389,Catalina Inn
70,70,saint-pierre,-21.3393,55.4781,26.82,78,0,1.54,RE,1675625567,Tropic Hotel
269,269,inhambane,-23.8650,35.3833,23.78,86,0,3.80,MZ,1675625659,Hotel de Inhambane
434,434,chokwe,-24.5333,32.9833,22.87,84,0,2.47,MZ,1675625735,Limpopo Hotel
472,472,salalah,17.0151,54.0924,23.05,73,0,2.06,OM,1675625747,Muscat International Hotel
537,537,nelson bay,-32.7167,152.1500,23.33,93,0,1.34,AU,1675625779,No hotel found'name'


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "StamenTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country"]
)
# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)